In [1]:
import pandas as pd
import numpy as np

In [19]:
categories=pd.read_csv('./data/item_categories.csv')
len(categories['item_category_name'].unique())
categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [29]:
categoriesd=categories.set_index('item_category_id').T.to_dict('list')

In [48]:
item_category_names=[]
for ri,rd in items.iterrows():
    item_category_names.append(categoriesd[rd['item_category_id']])
items['item_category_names']=item_category_names
items.head()

,item_name,item_id,item_category_id,item_category_names
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,[Кино - DVD]
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,[Программы - Для дома и офиса (Цифра)]
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,[Кино - DVD]
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,[Кино - DVD]
4,***КОРОБКА (СТЕКЛО) D,4,40,[Кино - DVD]


In [46]:
items=pd.read_csv('./data/items.csv')
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [8]:
shops=pd.read_csv('./data/shops.csv')
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
sales_train=pd.read_csv('./data/sales_train.csv')
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
